<h3> Steps </h3>

<ol>
    <li> Read Data</li>
    <li> Convert to Numpy Matrix and verify the dimension</li>

</ol>

In [1]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.layers import Dense, Input, Flatten, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D,GlobalMaxPooling1D
from keras.layers import Reshape, Dropout, Concatenate, LSTM,Bidirectional,BatchNormalization
from keras.layers import Flatten,Activation
from keras.models import Model
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
import os
import json
from IPython.core.display import display, HTML
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

Using TensorFlow backend.


In [2]:
dir = 'D:\Course Slides\Semester 3\Deep Learning for Speech\Project\ER'
PAD_ROWS=1800 #int(np.max(shapeList))26
PAD_COLS=26

In [21]:
from keras.layers import Dense, Input, Flatten, Conv2D,Conv1D, MaxPooling2D,MaxPooling1D

In [3]:
f = open(os.path.join(dir, 'train.json'), encoding="utf8")
data = json.load(f)

In [4]:
df = pd.DataFrame(data)
df= df.T
df.head()

,valence,activation,features
0,0,1,"[[5.502810676891276, 5.389630715979907, 5.8907..."
1,1,1,"[[5.059076172970736, 5.288492317702101, 4.2633..."
2,0,1,"[[4.218546271669202, 4.961436495859291, 3.6650..."
3,1,0,"[[4.650364321573866, 4.523905028353254, 5.0168..."
4,0,1,"[[3.900221957277269, 2.7325726489808124, 2.565..."


In [5]:
TOTAL_DATA=df.shape[0]

In [6]:
feat = np.zeros((TOTAL_DATA,PAD_ROWS,PAD_COLS),dtype=np.float32)

feature_set=pd.DataFrame(np.zeros((TOTAL_DATA,2),dtype=np.float32),columns=['features','label'])
feature_set['features']=df['features']
code=0
for i in tqdm_notebook(range(TOTAL_DATA)):
    
    [r,c]=np.array(df['features'][i]).shape
    feat[i,:r,:c]=np.array(normalize(df['features'][i]))
#     feat = np.array(feat.flatten())
#     feat=feat.transpose()
    feature_set['features'][i]=feat[i]
    v=df['valence'][i]
    a=df['activation'][i]
    
    if v==0 and a==0:
        code=0
    elif v==0 and a==1:
        code=1
    elif v==1 and a==0:
        code=2
    elif v==1 and a==1:
        code=3
    feature_set['label'][i]=code
    

    

In [7]:
print(feat.shape)
#feat

(7800, 1800, 26)


In [8]:
#labels = pd.get_dummies(feature_set['label']).values.tolist()
#features = np.matrix(feature_set['features'])

In [9]:
labels=feature_set['label']
labels=labels.astype(int)

In [10]:
X = feat
Y = np.array(labels)

<h6> Splitting the data into training and validation </h6>

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.10, random_state=96)

In [12]:
input_dim=X_train[0].shape
input_dim

(1800, 26)

In [13]:
# model = Sequential()

# #model.add(Dense(256,input_shape=input_dim, activation=tf.nn.relu))
# model.add(CuDNNLSTM(256, input_shape=input_dim, recurrent_dropout=0.2))
# model.add(Dropout(0.2))
# model.add(CuDNNLSTM(128, return_sequences=True, recurrent_dropout=0.2))
# model.add(Dropout(0.2))
# # model.add(Dense(128,activation=tf.nn.relu))
# # model.add(Dropout(0.5))
# # model.add(Dense(64,activation=tf.nn.relu))
# # model.add(Dropout(0.5))
# model.add(Dense(32,activation=tf.nn.relu))
# model.add(Dropout(0.5))
# # model.add(Dense(8,activation=tf.nn.relu))
# # model.add(Dropout(0.5))
# model.add(Dense(4, activation=tf.nn.softmax))
# model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])



<h6> Creating the Model </h6>

In [36]:
model = Sequential()

#model.add(Dense(256,input_shape=input_dim, activation=tf.nn.relu))
#model.add(Bidirectional(LSTM(64, dropout=0.4, recurrent_dropout=0.4,input_shape=input_dim, activation='relu', return_sequences=True)))
model.add(keras.layers.SimpleRNN(64, dropout=0.4, recurrent_dropout=0.2, input_shape=input_dim,return_sequences=False))
#model.add(Dropout(0.2))
#model.add(CuDNNLSTM(128))
#model.add(Dropout(0.2))
# model.add(Dense(128,activation=tf.nn.relu))
# model.add(Dropout(0.5))
# model.add(Dense(64,activation=tf.nn.relu))
# model.add(Dropout(0.5))

model.add(Dense(32,activation=tf.nn.relu))
model.add(Dropout(0.2))

# model.add(Dense(8,activation=tf.nn.relu))
# model.add(Dropout(0.5))

model.add(Dense(4, activation=tf.nn.softmax))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])



In [41]:
model = Sequential()
model.add(Conv1D(filters=16, kernel_size=3, activation='relu', input_shape=input_dim))
model.add(Conv1D(filters=16, kernel_size=6, activation='relu'))
model.add(Dropout(0.5))
model.add(Conv1D(filters=8, kernel_size=6, activation='relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [43]:

model.fit(X_train, y_train, epochs=200,verbose=2)  # train the model


Epoch 1/200
 - 7s - loss: 1.2803 - accuracy: 0.4050
Epoch 2/200
 - 7s - loss: 1.2611 - accuracy: 0.4080
Epoch 3/200
 - 7s - loss: 1.2498 - accuracy: 0.4130
Epoch 4/200
 - 7s - loss: 1.2383 - accuracy: 0.4118
Epoch 5/200
 - 7s - loss: 1.2289 - accuracy: 0.4140
Epoch 6/200
 - 8s - loss: 1.2285 - accuracy: 0.4192
Epoch 7/200
 - 7s - loss: 1.2281 - accuracy: 0.4168
Epoch 8/200
 - 7s - loss: 1.2173 - accuracy: 0.4291
Epoch 9/200
 - 7s - loss: 1.2084 - accuracy: 0.4321
Epoch 10/200
 - 7s - loss: 1.2090 - accuracy: 0.4322
Epoch 11/200
 - 7s - loss: 1.2010 - accuracy: 0.4409
Epoch 12/200
 - 7s - loss: 1.1923 - accuracy: 0.4427
Epoch 13/200
 - 7s - loss: 1.1808 - accuracy: 0.4489
Epoch 14/200
 - 7s - loss: 1.1740 - accuracy: 0.4556
Epoch 15/200
 - 7s - loss: 1.1687 - accuracy: 0.4605
Epoch 16/200
 - 7s - loss: 1.1619 - accuracy: 0.4588
Epoch 17/200
 - 7s - loss: 1.1517 - accuracy: 0.4677
Epoch 18/200
 - 7s - loss: 1.1455 - accuracy: 0.4718
Epoch 19/200
 - 7s - loss: 1.1345 - accuracy: 0.4759
Ep

Epoch 155/200
 - 7s - loss: 0.8657 - accuracy: 0.6064
Epoch 156/200
 - 7s - loss: 0.8655 - accuracy: 0.6084
Epoch 157/200
 - 8s - loss: 0.8612 - accuracy: 0.6174
Epoch 158/200
 - 8s - loss: 0.8590 - accuracy: 0.6075
Epoch 159/200
 - 7s - loss: 0.8542 - accuracy: 0.6222
Epoch 160/200
 - 7s - loss: 0.8566 - accuracy: 0.6134
Epoch 161/200
 - 7s - loss: 0.8560 - accuracy: 0.6135
Epoch 162/200
 - 7s - loss: 0.8538 - accuracy: 0.6142
Epoch 163/200
 - 7s - loss: 0.8488 - accuracy: 0.6098
Epoch 164/200
 - 7s - loss: 0.8572 - accuracy: 0.6121
Epoch 165/200
 - 7s - loss: 0.8578 - accuracy: 0.6115
Epoch 166/200
 - 7s - loss: 0.8589 - accuracy: 0.6051
Epoch 167/200
 - 7s - loss: 0.8382 - accuracy: 0.6232
Epoch 168/200
 - 7s - loss: 0.8564 - accuracy: 0.6134
Epoch 169/200
 - 7s - loss: 0.8527 - accuracy: 0.6165
Epoch 170/200
 - 7s - loss: 0.8580 - accuracy: 0.6158
Epoch 171/200
 - 7s - loss: 0.8416 - accuracy: 0.6189
Epoch 172/200
 - 7s - loss: 0.8466 - accuracy: 0.6171
Epoch 173/200
 - 7s - loss: 

In [32]:
#model.summary()

In [44]:
val_loss, val_acc = model.evaluate(X_test, y_test)  # evaluate the out of sample data with model
print(val_loss)  # model's loss (error)
print(val_acc)  # model's accuracy

780/780 [==============================] - 3s 4ms/step
1.5200869211783776
0.4076923131942749


In [ ]:
c=pd.DataFrame(b.features.values.tolist())

In [ ]:
from keras.models import model_from_json
model_json = model.to_json()
with open("model1.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model1.h5")
print("Saved model to disk")